# Oracle AI Vector Search：文档处理
Oracle AI Vector Search 专为人工智能 (AI) 工作负载而设计，它允许您根据语义而非关键字来查询数据。
Oracle AI Vector Search 的最大优势之一是能够在一个系统中将对非结构化数据的语义搜索与对业务数据的关系搜索相结合。
这不仅功能强大，而且效果显著，因为您无需添加专门的向量数据库，从而消除了多系统之间数据碎片化的烦恼。

此外，您的向量可以受益于 Oracle Database 所有最强大的功能，例如：

 * [分区支持](https://www.oracle.com/database/technologies/partitioning.html)
 * [Real Application Clusters 可扩展性](https://www.oracle.com/database/real-application-clusters/)
 * [Exadata 智能扫描](https://www.oracle.com/database/technologies/exadata/software/smartscan/)
 * [跨地理分布式数据库的分片处理](https://www.oracle.com/database/distributed-database/)
 * [事务](https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/transactions.html)
 * [并行 SQL](https://docs.oracle.com/en/database/oracle/oracle-database/21/vldbg/parallel-exec-intro.html#GUID-D28717E4-0F77-44F5-BB4E-234C31D4E4BA)
 * [灾难恢复](https://www.oracle.com/database/data-guard/)
 * [安全性](https://www.oracle.com/security/database-security/)
 * [Oracle 机器学习](https://www.oracle.com/artificial-intelligence/database-machine-learning/)
 * [Oracle 图数据库](https://www.oracle.com/database/integrated-graph-database/)
 * [Oracle Spatial and Graph](https://www.oracle.com/database/spatial/)
 * [Oracle 区块链](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_blockchain_table.html#GUID-B469E277-978E-4378-A8C1-26D3FF96C9A6)
 * [JSON](https://docs.oracle.com/en/database/oracle/oracle-database/23/adjsn/json-in-oracle-database.html)


本指南演示了如何通过 Oracle AI Vector Search 使用文档处理功能，分别使用 OracleDocLoader 和 OracleTextSplitter 加载和分块文档。

如果你刚开始使用 Oracle Database，可以考虑探索一下 [免费的 Oracle 23c AI](https://www.oracle.com/database/free/#resources)（请注意，原文是 Oracle 23c，这里保持了英文原文为 Oracle 23 AI，可能是一个笔误或者特指），它能很好地帮助你入门数据库环境的搭建。在使用数据库时，通常建议不要默认使用 system 用户；取而代之的是，你可以创建自己的用户来增强安全性和自定义功能。关于用户创建的详细步骤，请查阅我们的 [端到端指南](https://github.com/langchain-ai/langchain/blob/master/cookbook/oracleai_demo.ipynb)，该指南也展示了如何在 Oracle 中设置用户。此外，理解用户权限对于有效管理数据库安全至关重要。你可以在 Oracle 官方的 [用户账户和安全管理指南](https://docs.oracle.com/en/database/oracle/oracle-database/19/admqs/administering-user-accounts-and-security.html#GUID-36B21D72-1BBB-46C9-A0C9-F0D2A8591B8D) 中了解更多关于此主题的内容。

### 先决条件

请安装 Oracle Python Client 驱动程序，以便将 Langchain 与 Oracle AI Vector Search 结合使用。

In [ ]:
# pip install oracledb

### 连接到 Oracle 数据库
以下示例代码将展示如何连接到 Oracle 数据库。默认情况下，python-oracledb 以“Thin”模式运行，该模式直接连接到 Oracle 数据库。此模式不需要 Oracle Client 库。但是，当 python-oracledb 使用这些库时，会提供一些附加功能。当使用 Oracle Client 库时，python-oracledb 被称为处于“Thick”模式。两种模式都支持 Python Database API v2.0 规范的全面功能。请参阅以下[指南](https://python-oracledb.readthedocs.io/en/latest/user_guide/appendix_a.html#featuresummary)，其中介绍了每种模式支持的功能。如果无法使用 Thin 模式，您可能需要切换到 Thick 模式。

In [ ]:
import sys

import oracledb

# please update with your username, password, hostname and service_name
username = "<username>"
password = "<password>"
dsn = "<hostname>/<service_name>"

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

现在，让我们创建一个表并插入一些示例文档进行测试。

In [ ]:
try:
    cursor = conn.cursor()

    drop_table_sql = """drop table if exists demo_tab"""
    cursor.execute(drop_table_sql)

    create_table_sql = """create table demo_tab (id number, data clob)"""
    cursor.execute(create_table_sql)

    insert_row_sql = """insert into demo_tab values (:1, :2)"""
    rows_to_insert = [
        (
            1,
            "If the answer to any preceding questions is yes, then the database stops the search and allocates space from the specified tablespace; otherwise, space is allocated from the database default shared temporary tablespace.",
        ),
        (
            2,
            "A tablespace can be online (accessible) or offline (not accessible) whenever the database is open.\nA tablespace is usually online so that its data is available to users. The SYSTEM tablespace and temporary tablespaces cannot be taken offline.",
        ),
        (
            3,
            "The database stores LOBs differently from other data types. Creating a LOB column implicitly creates a LOB segment and a LOB index. The tablespace containing the LOB segment and LOB index, which are always stored together, may be different from the tablespace containing the table.\nSometimes the database can store small amounts of LOB data in the table itself rather than in a separate LOB segment.",
        ),
    ]
    cursor.executemany(insert_row_sql, rows_to_insert)

    conn.commit()

    print("Table created and populated.")
    cursor.close()
except Exception as e:
    print("Table creation failed.")
    cursor.close()
    conn.close()
    sys.exit(1)

### 加载文档

用户可以通过适当地配置加载器参数，灵活地从 Oracle 数据库、文件系统或两者同时加载文档。有关这些参数的详细信息，请参阅 [Oracle AI Vector Search Guide](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-73397E89-92FB-48ED-94BB-1AD960C4EA1F)。

使用 OracleDocLoader 的一个显著优势是它能够处理超过 150 种不同的文件格式，无需为不同文档类型使用多个加载器。支持格式的完整列表，请参阅 [Oracle Text Supported Document Formats](https://docs.oracle.com/en/database/oracle/oracle-database/23/ccref/oracle-text-supported-document-formats.html)。

以下是一个演示如何使用 OracleDocLoader 的代码片段：

In [ ]:
from langchain_community.document_loaders.oracleai import OracleDocLoader
from langchain_core.documents import Document

"""
# loading a local file
loader_params = {}
loader_params["file"] = "<file>"

# loading from a local directory
loader_params = {}
loader_params["dir"] = "<directory>"
"""

# loading from Oracle Database table
loader_params = {
    "owner": "<owner>",
    "tablename": "demo_tab",
    "colname": "data",
}

""" load the docs """
loader = OracleDocLoader(conn=conn, params=loader_params)
docs = loader.load()

""" verify """
print(f"Number of docs loaded: {len(docs)}")
# print(f"Document-0: {docs[0].page_content}") # content

### 分割文档
文档的大小各不相同，从小到大都有。用户通常倾向于将文档分割成更小的部分，以便生成嵌入。此分割过程提供了广泛的自定义选项。有关这些参数的详细信息，请参阅[Oracle AI Vector Search Guide](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-4E145629-7098-4C7C-804F-FC85D1F24240)。

下面是一个示例代码，说明如何实现这一点：

In [ ]:
from langchain_community.document_loaders.oracleai import OracleTextSplitter
from langchain_core.documents import Document

"""
# Some examples
# split by chars, max 500 chars
splitter_params = {"split": "chars", "max": 500, "normalize": "all"}

# split by words, max 100 words
splitter_params = {"split": "words", "max": 100, "normalize": "all"}

# split by sentence, max 20 sentences
splitter_params = {"split": "sentence", "max": 20, "normalize": "all"}
"""

# split by default parameters
splitter_params = {"normalize": "all"}

# get the splitter instance
splitter = OracleTextSplitter(conn=conn, params=splitter_params)

list_chunks = []
for doc in docs:
    chunks = splitter.split_text(doc.page_content)
    list_chunks.extend(chunks)

""" verify """
print(f"Number of Chunks: {len(list_chunks)}")
# print(f"Chunk-0: {list_chunks[0]}") # content

### 端到端演示

请参阅我们的完整演示指南[Oracle AI Vector Search End-to-End Demo Guide](https://github.com/langchain-ai/langchain/tree/master/cookbook/oracleai_demo.ipynb)，借助 Oracle AI Vector Search 构建端到端的 RAG 管道。